In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "4"

import numpy as np
import matplotlib.pyplot as plt
import re
from tqdm import tqdm
from pathlib import Path

import torch
from torch import nn
import torch.nn.functional as F



import sbi 
from sbi import analysis
from sbi import utils as utils
from sbi.inference import NLE, NPE
from sbi.neural_nets import posterior_nn, likelihood_nn 



/export/home/vgiusepp/miniconda3/envs/sbi_ds/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-05-01 20:09:02.967651: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-01 20:09:02.983992: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746122943.002048  966832 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746122943.007599  966832 cuda

In [3]:
#normalization function
        
def normalize(dataset, is_observable = False, dataset_name=None):
    if dataset_name is not None:
        mean = np.load(f'./params_compressor/normalization_for_compressor_{dataset_name}.npz')['mean']
        std = np.load(f'./params_compressor/normalization_for_compressor_{dataset_name}.npz')['std']
    if is_observable:
        # the shape in this case is (N, N_particles, 6)
        dataset_original_shape = dataset.shape
        normalized_dataset = dataset.reshape(-1, dataset.shape[-1])
        normalized_dataset = (normalized_dataset - mean)/ (std + 1e-8) 
        normalized_dataset = normalized_dataset.reshape(dataset_original_shape)
    else:
        normalized_dataset = (dataset - mean)/ (std + 1e-8) 

    return normalized_dataset

In [4]:
data_path = './data/data_NFW/'
pattern = re.compile(r"chunk_(\d+)\.npz")  # capture any number of digits
files = sorted(
    f for f in Path(data_path).glob("chunk_*.npz")
    if (m := pattern.fullmatch(f.name)) and int(m.group(1)) < 9500 and int(m.group(1)) > 0
)
theta_list, x_list, score_list = [], [], []

for f in files:
    data = np.load(f)
    theta_list.append(data["theta"])
    x_list.append(data["x"])
    score_list.append(data["score"]) 
    
dataset_theta = np.array(theta_list,).reshape(-1, 5)
dataset_y = np.array(x_list, ).reshape(-1, 10_000, 6)


#normalize the dataset
# dataset_theta = torch.tensor(normalize(dataset_theta, dataset_name='theta'))
# dataset_y = torch.tensor(normalize(dataset_y, is_observable=True, dataset_name='y'))

#rename
x = torch.tensor(dataset_y)
theta = torch.tensor(dataset_theta)

In [5]:
class DeepSetsEncoder(nn.Module):
    def __init__(self, input_dim=6, hidden_dim=128, output_dim=10):
        super().__init__()
        self.phi = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim),
        )

    def forward(self, x):
        """
        Args:
            x: Tensor of shape [N, D] (unbatched) or [B, N, D] (batched)
        Returns:
            summary vector of shape [output_dim] or [B, output_dim]
        """
        if x.dim() == 2:
            # Unbatched input: [N_particles, 6]
            x_phi = self.phi(x)                  # [N, output_dim]
            summary = x_phi.mean(dim=0)          # [output_dim]
        elif x.dim() == 3:
            # Batched input: [B, N_particles, 6]
            B, N, D = x.shape
            x_phi = self.phi(x.view(-1, D))      # [B * N, output_dim]
            x_phi = x_phi.view(B, N, -1)         # [B, N, output_dim]
            summary = x_phi.mean(dim=1)          # [B, output_dim]
        else:
            raise ValueError(f"Expected shape (N, D) or (B, N, D), got {x.shape}")
        
        return summary

In [20]:
prior_min = [0.5, 1e3, 0.1, 5e11, 1.0]
prior_max = [10.0, 1e5, 1.0, 1.5e12, 20.0]
prior = utils.torchutils.BoxUniform(
    low=torch.as_tensor(prior_min, device='cuda'), high=torch.as_tensor(prior_max, device='cuda')
)


# instantiate the custom embedding_net
embedding_net_custom = DeepSetsEncoder().to(device='cuda')

# Instantiate the conditional neural density estimator
# neural_posterior = likelihood_nn(model="maf", embedding_net=embedding_net_custom)
neural_posterior = posterior_nn(model="maf", embedding_net=embedding_net_custom)
inference = NPE(prior=prior, density_estimator=neural_posterior, device='cuda')

density_estimator = inference.append_simulations(theta.to(device='cuda'), x.to(device='cuda')).train()
posterior = inference.build_posterior(density_estimator)


/export/home/vgiusepp/miniconda3/envs/sbi_ds/lib/python3.12/site-packages/sbi/neural_nets/factory.py:334: UserWarning: The passed embedding net will be moved to cpu for
                        constructing the net building function.
  check_net_device(embedding_net, "cpu", embedding_net_warn_msg),
/export/home/vgiusepp/miniconda3/envs/sbi_ds/lib/python3.12/site-packages/sbi/inference/trainers/npe/npe_base.py:158: UserWarning: Data x has device 'cuda:0'. Moving x to the data_device 'cuda'. Training will proceed on device 'cuda'.
  theta, x = validate_theta_and_x(
/export/home/vgiusepp/miniconda3/envs/sbi_ds/lib/python3.12/site-packages/sbi/inference/trainers/npe/npe_base.py:158: UserWarning: Parameters theta has device 'cuda:0'. Moving theta to the data_device 'cuda'. Training will proceed on device 'cuda'.
  theta, x = validate_theta_and_x(


 Neural network successfully converged after 21 epochs.

In [24]:
data_path = './data/data_NFW/'
pattern = re.compile(r"chunk_(\d+)\.npz")  # capture any number of digits
files = sorted(
    f for f in Path(data_path).glob("chunk_*.npz")
    if (m := pattern.fullmatch(f.name)) and int(m.group(1)) > 9500 and int(m.group(1)) < 9503
)
theta_list, x_list, score_list = [], [], []

for f in files:
    data = np.load(f)
    theta_list.append(data["theta"])
    x_list.append(data["x"])
    score_list.append(data["score"]) 
    
dataset_theta = np.array(theta_list,).reshape(-1, 5)
dataset_y = np.array(x_list, ).reshape(-1, 10_000, 6)


#normalize the dataset
# dataset_theta = torch.tensor(normalize(dataset_theta, dataset_name='theta'))
# dataset_y = torch.tensor(normalize(dataset_y, is_observable=True, dataset_name='y'))

#rename
x = torch.tensor(dataset_y)
theta = torch.tensor(dataset_theta)

x_observed = x[0].to(device='cuda')
samples = posterior.sample((5000,), x=x_observed.to(device='cuda'), show_progress_bars=True)


# create the figure
# fig, ax = analysis.pairplot(
#     samples.cpu().numpy(),
#     points=true_parameter,
#     labels=["t_end", "Mtot_plummer", "a_plummer", "M_NFW", "r_s"],
#     # limits=np.stack([prior_min, prior_max], axis=1),
#     fig_kwargs=dict(
#         points_colors="r",
#         points_offdiag={"markersize": 6},
#     ),
#     figsize=(5, 5),
# )


Drawing 5000 posterior samples: 100%|██████████| 5000/5000 [00:00<00:00, 384826.78it/s]


In [25]:
samples

tensor([[1.4908e+00, 9.1778e+03, 1.2025e-01, 6.5511e+11, 9.0076e+00],
        [1.4908e+00, 9.1778e+03, 1.2025e-01, 6.5511e+11, 9.0076e+00],
        [1.4908e+00, 9.1778e+03, 1.2025e-01, 6.5511e+11, 9.0076e+00],
        ...,
        [1.4908e+00, 9.1778e+03, 1.2025e-01, 6.5511e+11, 9.0076e+00],
        [1.4908e+00, 9.1778e+03, 1.2025e-01, 6.5511e+11, 9.0076e+00],
        [1.4908e+00, 9.1778e+03, 1.2025e-01, 6.5511e+11, 9.0076e+00]],
       device='cuda:0')

In [26]:
true_parameter

tensor([1.4908e+00, 9.1778e+03, 1.2025e-01, 6.5511e+11, 9.0076e+00])

In [29]:
dataset_y[0]

array([[-2.3070920e-02, -2.3117030e-01,  1.6164832e+00,  4.0301746e+01,
        -1.5267348e-01,  4.9653131e-01],
       [ 1.6070962e-02, -1.0754041e-01,  1.5558944e+00,  3.8130051e+01,
        -7.5265393e-02,  4.0255699e-01],
       [-3.9953567e-02, -2.2966118e-01,  1.5971465e+00,  3.7623219e+01,
        -3.0063409e-03,  4.6757472e-01],
       ...,
       [-1.3626890e-01, -2.2150862e-01,  1.5936778e+00,  3.3417095e+01,
         6.7394689e-02,  2.0579502e-01],
       [-1.3131498e-01, -2.4196792e-01,  1.5895219e+00,  3.8400787e+01,
         5.7574641e-02,  4.4651896e-01],
       [-1.0581795e-02, -1.9501723e-01,  1.8260672e+00,  3.8036945e+01,
         1.8222505e-02,  4.2939904e-01]], dtype=float32)

In [33]:
dataset_y[1]

array([[-2.3070920e-02, -2.3117030e-01,  1.6164832e+00,  4.0301746e+01,
        -1.5267348e-01,  4.9653131e-01],
       [ 1.6070962e-02, -1.0754041e-01,  1.5558944e+00,  3.8130051e+01,
        -7.5265393e-02,  4.0255699e-01],
       [-3.9953567e-02, -2.2966118e-01,  1.5971465e+00,  3.7623219e+01,
        -3.0063409e-03,  4.6757472e-01],
       ...,
       [-1.3626890e-01, -2.2150862e-01,  1.5936778e+00,  3.3417095e+01,
         6.7394689e-02,  2.0579502e-01],
       [-1.3131498e-01, -2.4196792e-01,  1.5895219e+00,  3.8400787e+01,
         5.7574641e-02,  4.4651896e-01],
       [-1.0581795e-02, -1.9501723e-01,  1.8260672e+00,  3.8036945e+01,
         1.8222505e-02,  4.2939904e-01]], dtype=float32)